In [28]:
import json
import os
import re
import unicodedata
from collections import Counter

import pandas as pd

In [29]:
basedir= os.path.abspath(".")
datadir=os.path.join(basedir,"data")
processeddir = os.path.join(datadir,"processed")
traindta=os.path.join(datadir,"train.txt")
valpath= os.path.join(datadir,"val.txt")
testpath=os.path.join(datadir,"test.txt")
os.makedirs(processeddir, exist_ok=True)
traindta
valpath
testpath

'c:\\Users\\ravik\\Downloads\\Emotion_classification\\data\\test.txt'

In [30]:
def readdata(path):
    rows = []
    with open(path,"r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line:
                continue
            if ";" not in line:
                continue
            text,label =line.rsplit(";",1)
            text=text.strip()
            label=label.strip()
            if text and label:
                rows.append((text,label))
    return pd.DataFrame(rows,columns=["text", "label"])

In [31]:
train_df = readdata(traindta)


In [32]:
val_df = readdata(valpath)
test_df = readdata(testpath)
train_df.head(5)

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [33]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16000 non-null  object
 1   label   16000 non-null  object
dtypes: object(2)
memory usage: 250.1+ KB


In [34]:
df.isnull().sum()

text          0
label         0
clean_text    0
label_id      0
dtype: int64

In [24]:
df.duplicated().sum()
df[df.duplicated()]

,text,label,clean_text,label_id


In [25]:
train_df["label"].value_counts().len()

AttributeError: 'Series' object has no attribute 'len'

In [39]:
def prep(text):
    text=unicodedata.normalize("NFKC", text)
    text=text.lower()
    text=re.sub(r"[^a-z0-9\s']", " ", text)
    text=re.sub(r"\s+", " ", text).strip()
    return text

In [40]:
def clean(df):
    df=df.copy()
    df["clean_text"]=df["text"].map(prep)
    df = df[df["clean_text"].str.len() > 0]
    return df

train_df=clean(train_df)
val_df=clean(val_df)
test_df=clean(test_df)

train_df.head(5)

,text,label,label_id,clean_text
0,i didnt feel humiliated,sadness,4,i didnt feel humiliated
1,i can go from feeling so hopeless to so damned...,sadness,4,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wrong,anger,0,im grabbing a minute to post i feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,love,3,i am ever feeling nostalgic about the fireplac...
4,i am feeling grouchy,anger,0,i am feeling grouchy


In [41]:
labels=sorted(train_df["label"].unique())
label_to_id={}
id_to_label={}


In [ ]:

for idx,label in enumerate(labels):
    label_to_id[label]=idx
    id_to_label[idx]=label
for df in [train_df, val_df, test_df]:
    df["label_id"]=df["label"].apply(lambda x: label_to_id[x])
mapping_path=os.path.join(PROCESSED_DIR, "label_mapping.json")
with open(mapping_path, "w", encoding="utf-8") as f:
    json.dump(
        {"label_to_id": label_to_id,"id_to_label": id_to_label
        },
        f,
        indent=2
    )

label_to_id


{'anger': 0, 'fear': 1, 'joy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}

In [44]:
train_out=os.path.join(processeddir,"train.csv")
val_out=os.path.join(processeddir,"val.csv")
test_out=os.path.join(processeddir,"test.csv")
train_df[["clean_text", "label", "label_id"]].to_csv(train_out, index=False)
val_df[["clean_text", "label", "label_id"]].to_csv(val_out, index=False)
test_df[["clean_text", "label", "label_id"]].to_csv(test_out, index=False)

train_out, val_out, test_out

('c:\\Users\\ravik\\Downloads\\Emotion_classification\\data\\processed\\train.csv',
 'c:\\Users\\ravik\\Downloads\\Emotion_classification\\data\\processed\\val.csv',
 'c:\\Users\\ravik\\Downloads\\Emotion_classification\\data\\processed\\test.csv')

In [ ]:

counter = Counter()
for text in train_df["clean_text"]:
    counter.update(text.split())
vocab_size = len(counter)
most_common = counter.most_common(20)
vocab_size, most_common

(15212,
 [('i', 25859),
  ('feel', 11183),
  ('and', 9589),
  ('to', 8972),
  ('the', 8370),
  ('a', 6200),
  ('feeling', 5112),
  ('that', 5112),
  ('of', 4990),
  ('my', 4283),
  ('in', 3433),
  ('it', 3127),
  ('like', 2908),
  ('so', 2527),
  ('for', 2431),
  ('im', 2430),
  ('me', 2309),
  ('but', 2255),
  ('was', 2227),
  ('have', 2224)])